In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.optimizers import Adam
from keras.regularizers import l2
from imblearn.over_sampling import RandomOverSampler,SMOTE
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras import regularizers
from tensorflow.python.keras import backend as K
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Reshape
from keras.layers import MultiHeadAttention
from keras.layers import Input
from keras.layers import Lambda
from keras.layers import Concatenate,GlobalAveragePooling1D
from keras.layers import Masking, LSTM, TimeDistributed
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
import smote_variants as sv

In [2]:
data1 = pd.read_csv("dataset/AfterDeletejedit40.csv")
(row, cell) = data1.values.shape
data2=data1.values
X=np.array(data2).astype(float)  
#X =  X.CURE_SMOTE()

In [3]:
y=X[:,cell-1]
X=X[:,0:cell-1]
X_res, y_res = X, y
X_res.shape,y_res.shape
X_res.shape
X_res=X_res.reshape(row,cell-1,1)
X_train, y_train = X_res, y_res

In [4]:
data1 = pd.read_csv("dataset/AfterDeletejedit41.csv")
(row1, cell1) = data1.values.shape
data1.values.shape
data2=data1.values
X=np.array(data2).astype(float)  
#X =  X.CURE_SMOTE()

In [5]:
y=X[:,cell-1]
X=X[:,0:cell-1]
X_res, y_res = X, y
X_res.shape,y_res.shape
X_res.shape
X_res=X_res.reshape(row1,cell-1,1)
X_test, y_test = X_res, y_res

In [6]:
# 转换标签为独热编码
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((419, 20, 1), (283, 20, 1), (419, 2), (283, 2))

In [7]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
threshold = 0.5  

def mcc_metric(y_true, y_pred):
    predicted = tf.cast(tf.greater(y_pred, threshold), tf.float32)
    true_pos = tf.math.count_nonzero(predicted * y_true)
    true_neg = tf.math.count_nonzero((predicted - 1) * (y_true - 1))
    false_pos = tf.math.count_nonzero(predicted * (y_true - 1))
    false_neg = tf.math.count_nonzero((predicted - 1) * y_true)
    x = tf.cast((true_pos + false_pos) * (true_pos + false_neg) 
        * (true_neg + false_pos) * (true_neg + false_neg), tf.float32)
    return tf.cast((true_pos * true_neg) - (false_pos * false_neg), tf.float32) / tf.sqrt(x)


In [14]:
num_classes=2

def baseline_model():
    inputs = Input(shape=(cell-1, 1))
    
    # Convolutional layer
    conv = Conv1D(filters=12, kernel_size=4, activation='relu', kernel_regularizer='l2')(inputs)
    
    # Max pooling layer
    pool = MaxPooling1D(pool_size=2)(conv)
    
    # Multi-head attention layer
    multi_attention = MultiHeadAttention(num_heads=2, key_dim=2)(pool, pool)
    
    # Masking layer
    masked = Masking(mask_value=0.0)(multi_attention)
    
    # LSTM layer
    lstm = LSTM(128, return_sequences=True)(masked)
    
    # Time-distributed dense layer for self-supervised task
    self_supervised_task = TimeDistributed(Dense(1))(lstm)
    
    # Global pooling layers for self-supervised task output and multi-head attention output
    pooled_self_supervised_task = GlobalAveragePooling1D()(self_supervised_task)
    pooled_multi_attention = GlobalAveragePooling1D()(multi_attention)
    
    # Concatenating output of self-supervised task with the output of multi-head attention
    concatenated = Concatenate()([pooled_multi_attention, pooled_self_supervised_task])
    
    # Fully connected layer
    fc = Dense(21, activation='relu', kernel_regularizer='l2')(concatenated)
    
    # Output layer
    outputs = Dense(num_classes, activation='sigmoid', kernel_regularizer='l2')(fc)
    
    # Creating model
    model = Model(inputs=inputs, outputs=outputs)
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy', f1_m, recall_m, mcc_metric])
    
    return model

# Enable Eager execution
tf.config.run_functions_eagerly(True)


In [20]:
# build the model
model = baseline_model()
# Fit the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=64, batch_size=20,verbose=1)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[3]*100))


Epoch 1/64
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 212ms/step - accuracy: 0.4825 - f1_m: 0.5360 - loss: 3.6650 - mcc_metric: -0.0693 - recall_m: 0.6310 - val_accuracy: 0.2968 - val_f1_m: 0.2652 - val_loss: 1.1468 - val_mcc_metric: -0.2468 - val_recall_m: 0.2256
Epoch 2/64
21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 220ms/step - accuracy: 0.4548 - f1_m: 0.3962 - loss: 1.2759 - mcc_metric: -0.0657 - recall_m: 0.3566 - val_accuracy: 0.2615 - val_f1_m: 0.3176 - val_loss: 0.9849 - val_mcc_metric: -0.4520 - val_recall_m: 0.3400
Epoch 3/64
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 213ms/step - accuracy: 0.4981 - f1_m: 0.5056 - loss: 0.8977 - mcc_metric: 0.0185 - recall_m: 0.5088 - val_accuracy: 0.6431 - val_f1_m: 0.4445 - val_loss: 0.8911 - val_mcc_metric: nan - val_recall_m: 0.3567
Epoch 4/64
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 212ms/step - accuracy: 0.7022 - f1_m: 0.6721 - loss: 0.8257 - mcc_metric: 0.4441 - recall_m: 0.5921 - val_accuracy: 0.7420 - val_f1_m: 0.7552 - val_loss: 0.8329 - val_mcc_metric: 0.5221 - val_recall_m: 0.7467